In [6]:
import pandas as pd
import os
from pandas.core.arrays.period import timedelta
import numpy as np

In [7]:
def rename_prices(df):
    if 'PRICES' in df.columns:
        df.rename(columns={"PRICES": "datetime"}, inplace = True)
    else:
        print("There's no column PRICES.")


def dataformatting(df):
    #wide to long
    df = df.melt(id_vars=['datetime'], value_vars=df.columns[1:25]).sort_values(['datetime', 'variable'])
    df.reset_index(inplace=True, drop=True)
    
    #creating master time column, ulgy but works
    time = df['datetime'].copy()
    for d in range(len(df['datetime'])):
        time[d] = df['datetime'][d]+timedelta(hours = d%24) #decided not to go for the +1, so hour 1 is midnight, makes more sense, now it ends in 2009, otherwise the last measurement was 01.01.2010 00:00:00
    df['time'] = time
    
    #hour from string to int
    df['variable'] = df['variable'].map(lambda x:int(x[-2:]))
    
    #renaming, shullfing columns (not important)
    df.rename(columns={"datetime": "date", "variable": "hour", "value":"price"}, inplace = True)
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    
    return df

def drop_n_last_rows(df, n):
    df.drop(df.tail(1).index,inplace=True)


def conv(fld):
    if fld.endswith(b']'):
        return float(fld[:-1])
    elif fld.startswith(b'['):
        return float(fld[1:])
    else:
        return float(fld)
    
def load_validation_results(txt_file_name:str):
    val = pd.read_excel('./data/val_data/validate.xlsx')
    rename_prices(val)
    val = dataformatting(val)
    
    filename = txt_file_name
    
    # Drop the last row from validation data due to lack of action/reward taken when in that state 
    drop_n_last_rows(val, n=1)
    

    # To load the actions (reward/mkt_price)
    action_results = np.loadtxt(filename, delimiter=',', skiprows=3,max_rows = 1, dtype=float, converters=conv)
    val['action'] = action_results

    # To load the rewards
    reward_results = np.loadtxt(filename, delimiter=',', skiprows=1,max_rows = 1, dtype=float, converters=conv)
    val['reward'] = reward_results
    
    return val

val_data = load_validation_results('./archived_results/tabular_q/experiment_results(10k-earlystop).txt')

In [8]:
val_data

time       date  hour  price   action   reward
0     2010-01-01 00:00:00 2010-01-01     1  13.20  0.00000  0.00000
1     2010-01-01 01:00:00 2010-01-01     2  13.14  0.00000  0.00000
2     2010-01-01 02:00:00 2010-01-01     3  11.83  0.00000  0.00000
3     2010-01-01 03:00:00 2010-01-01     4   9.49  0.00000  0.00000
4     2010-01-01 04:00:00 2010-01-01     5   8.63 -0.00000 -0.00000
...                   ...        ...   ...    ...      ...      ...
17514 2011-12-31 18:00:00 2011-12-31    19  37.44  0.00000  0.00000
17515 2011-12-31 19:00:00 2011-12-31    20  34.56  0.00000  0.00000
17516 2011-12-31 20:00:00 2011-12-31    21  28.75  0.00000  0.00000
17517 2011-12-31 21:00:00 2011-12-31    22  25.79  1.32435  3.97305
17518 2011-12-31 22:00:00 2011-12-31    23  31.10  0.00000  0.00000

[17519 rows x 6 columns]